### Connecting to Postgresql

In [ ]:
pip install sqlalchemy

In [ ]:
pip install psycopg2-binary

In [4]:
%load_ext sql
from sqlalchemy import create_engine
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
pip install ipython-sql

In [5]:
%sql postgresql://postgres:@localhost:5432/superstoredb

In [ ]:
%%sql

CREATE TABLE Shipment (
    ShipmentID SERIAL PRIMARY KEY,
    ShipMode VARCHAR(14) NOT NULL
);

CREATE TABLE State (
    StateID SERIAL PRIMARY KEY,
    State VARCHAR(27) NOT NULL
);

CREATE TABLE Region (
    RegionID SERIAL PRIMARY KEY,
    Region VARCHAR(7) NOT NULL
);

CREATE TABLE City (
    CityID SERIAL PRIMARY KEY,
    City VARCHAR(45) NOT NULL
);

CREATE TABLE PostalCode (
    PostalCodeID SERIAL PRIMARY KEY,
    PostalCode VARCHAR(6) NOT NULL
);

CREATE TABLE Segment (
    SegmentID SERIAL PRIMARY KEY,
    Segment VARCHAR(11) NOT NULL
);

CREATE TABLE Country (
    CountryID SERIAL PRIMARY KEY,
    Country VARCHAR(13) NOT NULL
);

CREATE TABLE Customers (
    CustomerID VARCHAR(8) PRIMARY KEY,
    CustomerName VARCHAR(100) NOT NULL,
    CountryID INT,
    FOREIGN KEY (CountryID) REFERENCES Country(CountryID)
);



In [ ]:
%%sql

CREATE TABLE Category (
    CategoryID SERIAL PRIMARY KEY,
    Category VARCHAR(15)
);

CREATE TABLE SubCategory (
    SubCategoryID SERIAL PRIMARY KEY,
    SubCategory VARCHAR(50),
    CategoryID INT,
    FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID)
);

CREATE TABLE Product (
    ProductID VARCHAR(15) PRIMARY KEY,
    ProductName VARCHAR(255) NOT NULL,
    SubCategoryID INT,
    CategoryID INT,
    FOREIGN KEY (SubCategoryID) REFERENCES SubCategory(SubCategoryID),
    FOREIGN KEY (CategoryID) REFERENCES Category(CategoryID)
);

CREATE TABLE Orders (
    RowID INT PRIMARY KEY,
    OrderID VARCHAR(14) NOT NULL,
    OrderDate DATE,
    Sales DECIMAL(10, 5),
    Quantity INT,
    Discount DECIMAL(5, 2),
    Profit DECIMAL(10, 5),
    ShipDate DATE NOT NULL,
    CustomerID VARCHAR(8),
    ShipmentID INT,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID),
    FOREIGN KEY (ShipmentID) REFERENCES Shipment(ShipmentID)
);

In [ ]:
%%sql
INSERT INTO Country (Country)
SELECT DISTINCT Country
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM Country
    WHERE Country.Country = SuperStore.Country
);


In [ ]:
%%sql

INSERT INTO Customers (CustomerID, CustomerName, CountryID)
SELECT DISTINCT 
    SuperStore.CustomerID, 
    SuperStore.CustomerName, 
    Country.CountryID
FROM SuperStore
JOIN Country ON SuperStore.Country = Country.Country
WHERE NOT EXISTS (
    SELECT 1
    FROM Customers
    WHERE Customers.CustomerID = SuperStore.CustomerID
);


In [ ]:
%%sql

INSERT INTO Shipment (ShipDate, ShipMode)
SELECT DISTINCT ShipDate, ShipMode
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM Shipment
    WHERE Shipment.ShipMode = SuperStore.ShipMode
);

INSERT INTO state (state)
SELECT DISTINCT state
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM state
    WHERE state.state = SuperStore.state
);

INSERT INTO Region (Region)
SELECT DISTINCT Region
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM Region
    WHERE Region.Region = SuperStore.Region
);


INSERT INTO City (City)
SELECT DISTINCT City
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM City
    WHERE City.City = SuperStore.City
);


INSERT INTO PostalCode (PostalCode)
SELECT DISTINCT PostalCode
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM PostalCode
    WHERE PostalCode.PostalCode = SuperStore.PostalCode
);


INSERT INTO Segment (Segment)
SELECT DISTINCT Segment
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM Segment
    WHERE Segment.Segment = SuperStore.Segment
);



In [ ]:
%%sql

INSERT INTO Category (Category)
SELECT DISTINCT Category
FROM SuperStore
WHERE NOT EXISTS (
    SELECT 1
    FROM Category
    WHERE Category.Category = SuperStore.Category
);


INSERT INTO SubCategory (SubCategory, CategoryID)
SELECT DISTINCT 
    SuperStore.SubCategory, 
    Category.CategoryID
FROM SuperStore
JOIN Category ON Category.Category = SuperStore.Category
WHERE NOT EXISTS (
    SELECT 1
    FROM SubCategory
    WHERE SubCategory.SubCategory = SuperStore.SubCategory
      AND SubCategory.CategoryID = Category.CategoryID
);



In [ ]:

%%sql

INSERT INTO Product (ProductID, ProductName, SubCategoryID, CategoryID)
SELECT DISTINCT 
    SuperStore.ProductID, 
    SuperStore.ProductName,
    SubCategory.SubCategoryID,
    Category.CategoryID
FROM SuperStore
JOIN SubCategory ON SuperStore.SubCategory = SubCategory.SubCategory
JOIN Category ON SubCategory.CategoryID = Category.CategoryID
WHERE NOT EXISTS (
    SELECT 1
    FROM Product
    WHERE Product.ProductID = SuperStore.ProductID
);

INSERT INTO Orders (RowID, OrderID, OrderDate, Sales, Quantity, Discount, Profit, ShipDate, CustomerID, ShipmentID)
SELECT 
    SuperStore.RowID, 
    SuperStore.OrderID, 
    SuperStore.OrderDate, 
    SuperStore.Sales, 
    SuperStore.Quantity, 
    SuperStore.Discount, 
    SuperStore.Profit, 
    SuperStore.ShipDate, 
    SuperStore.CustomerID,
    Shipment.ShipmentID
FROM SuperStore
JOIN Shipment ON SuperStore.ShipMode = Shipment.ShipMode;


## Create Mapping Tables

In [ ]:
%%sql
CREATE TABLE StateMap (
    StateID INT,
    CustomerID VARCHAR(8),
    PRIMARY KEY (StateID, CustomerID),
    FOREIGN KEY (StateID) REFERENCES State(StateID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

CREATE TABLE RegionMap (
    RegionID INT,
    CustomerID VARCHAR(8),
    PRIMARY KEY (RegionID, CustomerID),
    FOREIGN KEY (RegionID) REFERENCES Region(RegionID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

CREATE TABLE CityMap (
    CityID INT,
    CustomerID VARCHAR(8),
    PRIMARY KEY (CityID, CustomerID),
    FOREIGN KEY (CityID) REFERENCES City(CityID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

CREATE TABLE PostalCodeMap (
    PostalCodeID INT,
    CustomerID VARCHAR(8),
    PRIMARY KEY (PostalCodeID, CustomerID),
    FOREIGN KEY (PostalCodeID) REFERENCES PostalCode(PostalCodeID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

CREATE TABLE SegmentMap (
    SegmentID INT,
    CustomerID VARCHAR(8),
    PRIMARY KEY (SegmentID, CustomerID),
    FOREIGN KEY (SegmentID) REFERENCES Segment(SegmentID),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

CREATE TABLE ProductMap (
    RowID INT,
    ProductID VARCHAR(15),
    PRIMARY KEY (RowID, ProductID),
    FOREIGN KEY (RowID) REFERENCES orders(RowID),
    FOREIGN KEY (ProductID) REFERENCES Product(ProductID)
);

### Inserting into Mapping Tables


In [ ]:
%%sql

INSERT INTO RegionMap (RegionID, CustomerID)
SELECT DISTINCT 
    Region.RegionID,
    Customers.CustomerID
FROM SuperStore
JOIN Region ON SuperStore.Region = Region.Region
JOIN Customers ON SuperStore.CustomerID = Customers.CustomerID
WHERE NOT EXISTS (
    SELECT 1
    FROM RegionMap
    WHERE RegionMap.RegionID = Region.RegionID
      AND RegionMap.CustomerID = Customers.CustomerID
);


INSERT INTO CityMap (CityID, CustomerID)
SELECT DISTINCT 
    City.CityID,
    Customers.CustomerID
FROM SuperStore
JOIN City ON SuperStore.City = City.City
JOIN Customers ON SuperStore.CustomerID = Customers.CustomerID
WHERE NOT EXISTS (
    SELECT 1
    FROM CityMap
    WHERE CityMap.CityID = City.CityID
      AND CityMap.CustomerID = Customers.CustomerID
);


INSERT INTO PostalCodeMap (PostalCodeID, CustomerID)
SELECT DISTINCT 
    PostalCode.PostalCodeID,
    Customers.CustomerID
FROM SuperStore
JOIN PostalCode ON SuperStore.PostalCode = PostalCode.PostalCode
JOIN Customers ON SuperStore.CustomerID = Customers.CustomerID
WHERE NOT EXISTS (
    SELECT 1
    FROM PostalCodeMap
    WHERE PostalCodeMap.PostalCodeID = PostalCode.PostalCodeID
      AND PostalCodeMap.CustomerID = Customers.CustomerID
);


INSERT INTO SegmentMap (SegmentID, CustomerID)
SELECT DISTINCT 
    Segment.SegmentID,
    Customers.CustomerID
FROM SuperStore
JOIN Segment ON SuperStore.Segment = Segment.Segment
JOIN Customers ON SuperStore.CustomerID = Customers.CustomerID
WHERE NOT EXISTS (
    SELECT 1
    FROM SegmentMap
    WHERE SegmentMap.SegmentID = Segment.SegmentID
      AND SegmentMap.CustomerID = Customers.CustomerID
);


INSERT INTO ProductMap (RowID, ProductID)
SELECT DISTINCT 
    orders.RowID,
    Product.ProductID
FROM SuperStore
JOIN orders ON SuperStore.RowID = orders.RowID
JOIN Product ON SuperStore.ProductID = Product.ProductID
WHERE NOT EXISTS (
    SELECT 1
    FROM ProductMap
    WHERE ProductMap.RowID = orders.RowID
      AND ProductMap.ProductID = Product.ProductID
);

INSERT INTO StateMap (StateID, CustomerID)
SELECT DISTINCT 
    state.StateID,
    Customers.CustomerID
FROM SuperStore
JOIN state ON SuperStore.state = state.State
JOIN Customers ON SuperStore.CustomerID = Customers.CustomerID
WHERE NOT EXISTS (
    SELECT 1
    FROM StateMap
    WHERE StateMap.StateID = state.StateID
      AND StateMap.CustomerID = Customers.CustomerID
);
